In [9]:
import pandas as pd
from datetime import datetime, timedelta

In [20]:
pd.__version__

'2.2.2'

In [23]:
!python3.10 --version

Python 3.10.16


In [24]:
ticker_file = '/Users/liuda/Local/data/trading/data/stocks/ticker/history/AMZN.csv'

In [25]:
ticker_data = pd.read_csv(ticker_file, sep=",")

In [26]:
ticker_data['TICKER'] = 'AMZN'
ticker_data['Adj High'] = ticker_data['High'] + ticker_data['Adj Close'] - ticker_data['Close']
ticker_data['Adj Low'] = ticker_data['Low'] + ticker_data['Adj Close'] - ticker_data['Close']
ticker_data['Adj Open'] = ticker_data['Open'] + ticker_data['Adj Close'] - ticker_data['Close']

In [28]:
ticker_data = ticker_data.sort_values(by="Date").reset_index(drop=True)
ticker_data['qt'] = ticker_data['Date'].apply(pd.to_datetime).dt.to_period('Q').astype(str).str.replace('Q', 'q')

hedge_filter = pd.read_csv('/Users/liuda/Local/data/trading/data/output/filter/hedge_filters.csv')
# ticker_data = join_hedge_filter(ticker_data, hedge_filter)

ticker_data = ticker_data.sort_values(by='Date').reset_index(drop=True)
ticker_data['Date'] = pd.to_datetime(ticker_data['Date'])
ticker_data.set_index('Date', inplace=True)

In [29]:
resample_rules = [("W", "weekly"), ("ME", "monthly")]

In [30]:
ticker_data_fill_30_day = ticker_data.copy(deep=True)
future_dates = pd.date_range(start=ticker_data_fill_30_day.index[-1] + pd.Timedelta(days=1), periods=30)
ticker_data_fill_30_day = ticker_data_fill_30_day.reindex(ticker_data_fill_30_day.index.union(future_dates))
ticker_data_fill_30_day = ticker_data_fill_30_day.ffill()
ticker_data_fill_30_day.rename_axis('Date', inplace=True)   # index列重命名为Date，后续Join要使用


In [31]:
rule, label = resample_rules[0]

In [32]:
rule, label

('W', 'weekly')

In [33]:
ticker_data_fill_30_day.resample(   # 从fill 30 day 采样，避免尾部值缺失
    rule, label="left", offset="1D"
).agg(
    {
        "Open": "first",
        "High": "max",
        "Low": "min",
        "Adj High": "max",
        "Adj Open": "first",
        "Adj Low": "min",
        "Adj Close": "last",
        "Volume": "sum",
    }
)

,Open,High,Low,Adj High,Adj Open,Adj Low,Adj Close,Volume
Date,,,,,,,,
2010-03-28,6.7685,6.909500,6.559000,6.909500,6.7685,6.559000,6.590500,4.841200e+08
2010-04-04,6.6425,7.066500,6.539000,7.066500,6.6425,6.539000,7.003000,7.682480e+08
2010-04-11,7.0000,7.358500,6.956000,7.358500,7.0000,6.956000,7.108500,6.871500e+08
2010-04-18,7.1175,7.554500,6.956500,7.554500,7.1175,6.956500,7.181500,1.036982e+09
2010-04-25,7.1600,7.386500,6.845500,7.386500,7.1600,6.845500,6.855000,7.924060e+08
...,...,...,...,...,...,...,...,...
2025-03-23,200.0000,203.639999,199.949997,203.639999,200.0000,199.949997,203.259995,2.908507e+08
2025-03-30,200.0000,203.639999,199.949997,203.639999,200.0000,199.949997,203.259995,2.908507e+08
2025-04-06,200.0000,203.639999,199.949997,203.639999,200.0000,199.949997,203.259995,2.908507e+08


In [1]:
import pandas as pd

In [5]:
input_file = '/Users/liuda/Desktop/QQQ.csv'

df = pd.read_csv(input_file)
print(df.loc[df['Date'].idxmax()][['TICKER', 'Adj Close', 'SMA_5_day', 'SMA_20_day']])

TICKER               QQQ
Adj Close     484.380005
SMA_5_day      485.43407
SMA_20_day    485.340225
Name: 3770, dtype: object


In [6]:
input_file = '/Users/liuda/Desktop/SPY.csv'

df = pd.read_csv(input_file)
print(df.loc[df['Date'].idxmax()][['TICKER', 'Adj Close', 'SMA_5_day', 'SMA_20_day']])

TICKER               SPY
Adj Close     568.590027
SMA_5_day     569.180811
SMA_20_day    568.834583
Name: 3770, dtype: object
